#### Flight Radar Pipeline

In [1]:
!pip install ydata-synthetic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for easydict: filename=easydict-1.10-py3-none-any.whl size=6491 sha256=4f17eac582bcbef6773f7524cc9cf3befab2cc72a04335700f099a4ef6549be6
  Stored in directory: /root/.cache/pip/wheels/b7/e3/8c/f814820ac7ef5fd96f38759ce0d3dc51166a3169008b31da53
Successfully built easydict
  Attempting uninstall: easydict
    Found existing installation: easydict 1.13
    Uninstalling easydict-1.13:
      Successfully uninstalled easydict-1.13
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.

In [20]:
# import all necessary libraries
import os
import pandas as pd
import numpy as np
# from ydata_synthetic.synthesizers.regular import RegularSynthesizer
# from ydata_synthetic.synthesizers import ModelParameters,TrainParameters
import boto3
import requests
from dotenv import load_dotenv

# load environment variables
load_dotenv()


True

In [3]:
# create a function to scrape the data
def scrape_data():
    # api endpoint
    endpoint = "https://flightradar243.p.rapidapi.com/v1/flights/most-tracked"

    token = "bearer-token"
    # set the headers
    headers = {
    "X-RapidAPI-Key": "79a1b32ed1mshac16a51ba8187b1p162d29jsn2549136b9466",
    "X-RapidAPI-Host": "flightradar243.p.rapidapi.com"
      }

    # Send a get request to the URL
    response = requests.get(endpoint,headers=headers)
    try:
        if response.status_code == 200:
            print("successfuly connected")
            print(response.status_code)
            # parse response as json
            data = response.json()
            return data

        else:
            print("Connection unsuccessful")
            print(response.status_code)

    except Exception as e:
        print(e)


# store the return data
json_data = scrape_data()

# check the structure of the json data
json_data

successfuly connected
200


{'status': 'success',
 'data': {'version': '0.3.8',
  'update_time': 1716360633.232379,
  'data': [{'flight_id': '355288a8',
    'flight': 'SQ321',
    'callsign': 'SIA321',
    'squawk': None,
    'clicks': 1176,
    'from_iata': 'LHR',
    'from_city': 'London',
    'to_iata': 'SIN',
    'to_city': 'Singapore',
    'model': 'B77W',
    'type': 'Boeing 777-312(ER)'},
   {'flight_id': '3553cbd8',
    'flight': 'CTM1018',
    'callsign': 'CTM1018',
    'squawk': None,
    'clicks': 952,
    'from_iata': 'NOU',
    'from_city': 'Noumea',
    'to_iata': None,
    'to_city': None,
    'model': 'A332',
    'type': 'Airbus A330-243'},
   {'flight_id': '3553a103',
    'flight': 'RSD974',
    'callsign': 'RSD974',
    'squawk': None,
    'clicks': 636,
    'from_iata': 'VKO',
    'from_city': 'Moscow',
    'to_iata': None,
    'to_city': None,
    'model': 'T214',
    'type': 'Tupolev Tu-214PU'},
   {'flight_id': '354fc50c',
    'flight': None,
    'callsign': 'FORTE11',
    'squawk': None,
  

In [5]:
# take the data from the json_data and store it as flight_data
if json_data and 'data' in json_data:
    flight_data = json_data['data']['data']
else:
    print("No data found in the JSON response.")

,flight_id,flight,callsign,squawk,clicks,from_iata,from_city,to_iata,to_city,model,type
0,355288a8,SQ321,SIA321,None,1176,LHR,London,SIN,Singapore,B77W,Boeing 777-312(ER)
1,3553cbd8,CTM1018,CTM1018,None,952,NOU,Noumea,None,None,A332,Airbus A330-243
2,3553a103,RSD974,RSD974,None,636,VKO,Moscow,None,None,T214,Tupolev Tu-214PU
3,354fc50c,None,FORTE11,None,520,None,None,None,None,Q4,Northrop Grumman RQ-4B Global Hawk
4,3552e106,EK21,UAE1KM,None,292,DXB,Dubai,MAN,Manchester,A388,Airbus A380-842
5,35535249,PG711,BKP711,None,280,BKK,Bangkok,MLE,Male,A319,Airbus A319-132
6,3553bd93,Q2443,DQA443,None,268,HDK,Kulhudhuffushi,MLE,Male,AT76,ATR 72-600
7,3552d5b4,QF6036,QFA6036,None,260,MNL,Manila,SYD,Sydney,A332,Airbus A330-203
8,3553b89d,SU2139,AFL2139,None,244,IST,Istanbul,SVO,Moscow,B738,Boeing 737-8LJ
9,3553b859,JT126,LNI126,None,200,CGK,Jakarta,TJQ,Tanjung Pandan,B738,Boeing 737-8GP


#### Generating Synthetic Data

In [16]:
num_cols = ['clicks']
cat_cols = ['flight_id', 'flight', 'callsign', 'squawk', 'from_iata', 'from_city', 'to_iata', 'to_city', 'model', 'type']

In [11]:
# defining parameters for the package

batch_size = 50
epochs = 500 + 1
learning_rate = 2e-4
beta_1 = 0.5
beta_2 = 0.9

ctgan_args = ModelParameters(
    batch_size=batch_size,
    lr=learning_rate,
    betas=(beta_1,beta_2))

train_args = TrainParameters(
    epochs=epochs,
)


In [17]:
flights_df = pd.DataFrame(flight_data)
synth = RegularSynthesizer(modelname="ctgan", model_parameters=ctgan_args)
synth.fit(flights_df, train_arguments=train_args, num_cols=num_cols, cat_cols=cat_cols)


Epoch: 0 | critic_loss: 5.585970878601074 | generator_loss: 1.9797704219818115
Epoch: 1 | critic_loss: 5.838409900665283 | generator_loss: 2.1415441036224365
Epoch: 2 | critic_loss: 5.420535564422607 | generator_loss: 2.217543601989746
Epoch: 3 | critic_loss: 5.460684776306152 | generator_loss: 2.1301352977752686
Epoch: 4 | critic_loss: 5.263073921203613 | generator_loss: 2.0130257606506348
Epoch: 5 | critic_loss: 5.038912773132324 | generator_loss: 2.113956928253174
Epoch: 6 | critic_loss: 5.151932716369629 | generator_loss: 2.0430591106414795
Epoch: 7 | critic_loss: 4.897109508514404 | generator_loss: 2.087378978729248
Epoch: 8 | critic_loss: 4.890458583831787 | generator_loss: 2.006443738937378
Epoch: 9 | critic_loss: 4.79616641998291 | generator_loss: 2.0690808296203613
Epoch: 10 | critic_loss: 4.471642971038818 | generator_loss: 2.1744134426116943
Epoch: 11 | critic_loss: 4.2796244621276855 | generator_loss: 1.9522699117660522
Epoch: 12 | critic_loss: 4.50684928894043 | generator_

In [18]:
synth_data = synth.sample(10000)
synth_data

,flight_id,flight,callsign,squawk,clicks,from_iata,from_city,to_iata,to_city,model,type
0,35535249,CTM1018,FORTE11,None,1496,NOU,Manila,MLE,None,A332,Airbus A380-842
1,355288a8,CTM1018,DQA443,None,-315,None,Bangkok,MLE,Moscow,Q4,Tupolev Tu-214PU
2,3553b859,JT126,BKP711,None,-213,BKK,Manila,SVO,Tanjung Pandan,AT76,Boeing 777-312(ER)
3,3553b89d,SQ321,CTM1018,None,105,IST,London,MLE,Tanjung Pandan,B77W,Northrop Grumman RQ-4B Global Hawk
4,3553cbd8,EK21,SIA321,None,-309,HDK,Istanbul,None,Male,T214,Northrop Grumman RQ-4B Global Hawk
...,...,...,...,...,...,...,...,...,...,...,...
9995,3552e106,CTM1018,RSD974,None,13,MNL,London,SYD,Moscow,A332,Airbus A380-842
9996,3553b859,SU2139,CTM1018,None,954,CGK,London,None,None,A319,Boeing 777-312(ER)
9997,3553cbd8,PG711,RSD974,None,109,CGK,Bangkok,SIN,Sydney,T214,Airbus A330-243
9998,3553b859,Q2443,AFL2139,None,-124,HDK,Dubai,TJQ,None,B738,Airbus A330-243


In [19]:
synth_df = pd.DataFrame(synth_data)
synth_df

,flight_id,flight,callsign,squawk,clicks,from_iata,from_city,to_iata,to_city,model,type
0,35535249,CTM1018,FORTE11,None,1496,NOU,Manila,MLE,None,A332,Airbus A380-842
1,355288a8,CTM1018,DQA443,None,-315,None,Bangkok,MLE,Moscow,Q4,Tupolev Tu-214PU
2,3553b859,JT126,BKP711,None,-213,BKK,Manila,SVO,Tanjung Pandan,AT76,Boeing 777-312(ER)
3,3553b89d,SQ321,CTM1018,None,105,IST,London,MLE,Tanjung Pandan,B77W,Northrop Grumman RQ-4B Global Hawk
4,3553cbd8,EK21,SIA321,None,-309,HDK,Istanbul,None,Male,T214,Northrop Grumman RQ-4B Global Hawk
...,...,...,...,...,...,...,...,...,...,...,...
9995,3552e106,CTM1018,RSD974,None,13,MNL,London,SYD,Moscow,A332,Airbus A380-842
9996,3553b859,SU2139,CTM1018,None,954,CGK,London,None,None,A319,Boeing 777-312(ER)
9997,3553cbd8,PG711,RSD974,None,109,CGK,Bangkok,SIN,Sydney,T214,Airbus A330-243
9998,3553b859,Q2443,AFL2139,None,-124,HDK,Dubai,TJQ,None,B738,Airbus A330-243


In [20]:
# convert to csv
synth_df.to_csv('synthetic_data.csv', index=False)

#### Persist Data to s3 Bucket

In [22]:
def upload_csv_to_s3(file_name, bucket_name, object_name=None):
    """
    Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket_name: S3 bucket to upload to
    :param object_name: S3 object name. If not specified, then file_name is used
    :return: True if file was uploaded, else False
    """
    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Create a session using the same credentials as AWS CLI
    session = boto3.Session(
        aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),  
        aws_secret_access_key= os.getenv('AWS_SECRET_ACCESS_KEY'),  
        region_name = os.getenv('AWS_DEFAULT_REGION')
    )

    # Upload the file
    s3_client = session.client('s3')
    try:
        s3_client.upload_file(file_name, bucket_name, object_name)
        print(f"Successfully uploaded {file_name} to {bucket_name}/{object_name}")
    except boto3.exceptions.S3UploadFailedError as e:
        print(f"Failed to upload {file_name} to {bucket_name}/{object_name}: {e}")
        return False
    except Exception as e:
        print(f"An error occurred: {e}")
        return False
    return True

def main():
    file_name = "../data/synthetic_flight_data.csv"  # Ensure this path is correct
    bucket_name = "trestleacademybucket"  # Your S3 bucket name
    object_name = "data-pipeline/synthetic_flight_data.csv"  # Path in the S3 bucket

    success = upload_csv_to_s3(file_name, bucket_name, object_name)
    if success:
        print("File uploaded successfully")
    else:
        print("File upload failed")

if __name__ == "__main__":
    main()


Successfully uploaded ../data/synthetic_flight_data.csv to trestleacademybucket/data-pipeline/synthetic_flight_data.csv
File uploaded successfully
